In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from sklearn.metrics import confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Circle

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
file_name = repo_path / 'data/covert-search/large-field/p4_data_AZ_t1.mat'
data = loadmat(file_name)
print(data.keys())

In [ ]:
taget_amplitude = data['targetAmplitude']
target_location = data['tLocation']
spot_center = data['spotCenters']
human_response = data['hResponse']
n_location = data['nLocations'][0][0]
spot_diameter = data['spotLength'][0][0]
stimulus_design_size= data['totalLength'][0][0]
monitor_width = data['monitorPx'][0][0]
monitor_height = data['monitorPx'][0][1]

In [ ]:
shifted_spot_center = spot_center.copy()
shifted_spot_center[:,0] += (monitor_height - stimulus_design_size)//2
shifted_spot_center[:,1] += (monitor_width - stimulus_design_size)//2

accurate_response = target_location == human_response

In [ ]:
stimulus_region = np.zeros((monitor_height, monitor_width))
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for index_location in range(n_location-1):
    spot_region = (pixel_row - shifted_spot_center[index_location, 0])**2+(pixel_col - shifted_spot_center[index_location, 1])**2<= spot_diameter**2/4
    stimulus_region[spot_region] = 1

In [ ]:
fig, ax = plt.subplots()
ax.imshow(stimulus_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center')

circle = Circle((monitor_width//2, monitor_height//2), spot_diameter*1.5, edgecolor='r', facecolor='none', linewidth=3)
ax.add_patch(circle)

In [ ]:
array_eccentral_distance = np.zeros((n_location-1,))

for index_location in range(n_location-1):
    array_eccentral_distance[index_location] = np.sqrt((spot_center[index_location, 0] - stimulus_design_size//2)**2+(spot_center[index_location, 1] - stimulus_design_size//2)**2)

In [ ]:
eccentral_distance_sorted_indexes = np.argsort(array_eccentral_distance)
extra_eccentral_distance_sorted_indexes = np.insert(eccentral_distance_sorted_indexes+1,0,0)
print(extra_eccentral_distance_sorted_indexes)

In [ ]:
accurate_response = target_location == human_response
confusion_mat = confusion_matrix(human_response.flatten(), target_location.flatten(), labels=range(n_location))[extra_eccentral_distance_sorted_indexes, :][:, extra_eccentral_distance_sorted_indexes]
log_confusion_mat = np.log(confusion_mat+1)
array_index_location = np.arange(n_location)[extra_eccentral_distance_sorted_indexes]

In [ ]:
fig, ax = plt.subplots(figsize=(15,12))
im = ax.imshow(log_confusion_mat)

for index_target_location in array_index_location:
    for index_human_response in array_index_location:
        ax.text(index_human_response, index_target_location, f'{confusion_mat[index_target_location, index_human_response]:.0f}', ha='center', va='center')

ax.set(xticks=np.arange(n_location), yticks=np.arange(n_location), xticklabels=array_index_location, yticklabels=array_index_location, xlabel='Target location sorted by eccentral distance', ylabel='Human response sorted by eccentral distance')

plt.show()

In [ ]:
indixes_near = np.arange(1,n_location)[array_eccentral_distance < spot_diameter*1.5]
indixes_far = np.arange(1,n_location)[array_eccentral_distance > spot_diameter*1.5]
print(indixes_near, indixes_far)

In [ ]:
accuracy_near = accurate_response[np.isin(target_location, indixes_near)].mean()
print(accuracy_near)